Incorporar fotos estáticas de *view_fac_maestro_portafolio_factoring*
en las fechas usadas para calcular los esquemas comisionales

In [32]:
mes_incorporar = '202512' # formato YYYYMM

In [33]:
import pandas as pd
!pip install boto3
import boto3
import json
import io
import os
import numpy as np
!pip install pyathena
from pyathena import connect

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
#%% Credenciales de AmazonAthena
with open('/content/credenciales actualizado.txt') as f:
    creds = json.load(f)

conn = connect(
    aws_access_key_id     = creds["AccessKeyId"],
    aws_secret_access_key = creds["SecretAccessKey"],
    aws_session_token     = creds["SessionToken"],
    s3_staging_dir        = creds["s3_staging_dir"],
    region_name           = creds["region_name"]

    )

In [35]:
# obteniendo datos actuales
query = '''
    SELECT * FROM  prod_datalake_sandbox.ba__fac_monthly_portfolio

'''
cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_monthly_portfolio = pd.DataFrame(resultados, columns=column_names)
print(df_monthly_portfolio.shape)

# filtración por si hay que volver a actualizar el mismo mes
df_monthly_portfolio = df_monthly_portfolio[df_monthly_portfolio['codmes'] != mes_incorporar]

# columnas para asegurar el rename de la nueva base
columnas_base = df_monthly_portfolio.columns

(10586, 81)


In [36]:
dtype_dict = {
    'codigo_subasta': str,
    'cierre_fecha': str,
    'codmes': str,
    'producto': str,
    'ruc_cliente': str,
    'nombre_cliente': str,
    'ruc_proveedor': str,
    'nombre_proveedor': str,
    'fecha_desembolso': str,
    'moneda': str,
    'monto_neto_total_pendiente': float,
    'monto_financiado': float,
    'plazo': float,
    'tasa_empresario': float,
    'monto_adelanto': float,
    'ejecutivo': str,
    'monto_total_pagado': float,
    'capital_pagado': float,
    'interes_pagado': float,
    'ultima_fecha_pago': str,
    'fecha_vencimiento': str,
    'q_desembolso': float,
    'm_desembolso': float,
    'saldo_capital': float,
    'saldo_monto': float,
    'status': str,
    'dias_atraso': float,
    'm_desembolso_soles': float,
    'saldo_capital_soles': float,
    'monto_financiado_soles': float,
    'tipo_cambio': float,
    'codmes_desembolso': int,
    'par1_ms': float,
    'par15_ms': float,
    'par30_ms': float,
    'par60_ms': float,
    'par90_ms': float,
    'par120_ms': float,
    'par180_ms': float,
    'par180_m': float,
    'par360_ms': float,
    'flag_origen_operacion': str,
    'new_clients': int,
    'new_providers': int,
    'fuente': str,
    'propietario_negocio': str,
    'tipificacion_operativa': str,
    'confirmaron_todas_las_facturas_en_cavali_': str,
    'correo_de_confirmacion': str,
    'comision_estructuracion': float,
    'comision_estructuracion_solarizado': float,
    'tasa_all_in': float,
    'utilidad_soles': float,
    'fecha': str,
    'day': int,
    'month': int,
    'year': int,
    'day_name': str,
    'week': int,
    'quarter_name': str,
    'semester': int,
    'day_of_week': int,
    'day_of_year': int,
    'is_weekend': str,
    'is_holiday': str,
    'fiscal_year': int,
    'fiscal_month': int,
    'fiscal_quarter': str,
    'year_month': str,
    'riesgo': str,
    'financiado_soles_gestora': float,
    'financiado_soles_fp': float,
    'financiado_soles_crowd': float,
    'saldo_gestora': float,
    'saldo_fp': float,
    'saldo_crowd': float,
    'grupo_economico': str,
    'actividad_economica': str,
    'sector_proveedor': str,
    'sector_cliente': str
}

# df_monthly_portfolio = df_monthly_portfolio.astype(dtype_dict, errors='ignore')
df_monthly_portfolio = df_monthly_portfolio.astype(dtype_dict)

df_monthly_portfolio['cierre_fecha']      = pd.to_datetime(df_monthly_portfolio['cierre_fecha']).dt.date
df_monthly_portfolio['fecha_desembolso']  = pd.to_datetime(df_monthly_portfolio['fecha_desembolso']).dt.date

# Convierte None explícitamente a NaN para que pandas los reconozca
df_monthly_portfolio['ultima_fecha_pago'] = df_monthly_portfolio['ultima_fecha_pago'].replace({None: pd.NA})
# # Luego convierte a datetime
df_monthly_portfolio['ultima_fecha_pago'] = pd.to_datetime(df_monthly_portfolio['ultima_fecha_pago'], errors='coerce')
# # Y finalmente deja solo la fecha (sin hora)
df_monthly_portfolio['ultima_fecha_pago'] = df_monthly_portfolio['ultima_fecha_pago'].dt.date

df_monthly_portfolio['fecha_vencimiento'] = pd.to_datetime(df_monthly_portfolio['fecha_vencimiento']).dt.date
df_monthly_portfolio['fecha']             = pd.to_datetime(df_monthly_portfolio['fecha']).dt.date

df_monthly_portfolio['_timestamp'] = pd.to_datetime(df_monthly_portfolio['_timestamp'])

/tmp/ipython-input-534654023.py:93: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_monthly_portfolio['ultima_fecha_pago'] = pd.to_datetime(df_monthly_portfolio['ultima_fecha_pago'], errors='coerce')


In [37]:
# print(df_nuevos_datos.columns)


# print(columnas_base)

In [38]:
# obteniendo los datos del nuevo mes:
query = f'''
SELECT * FROM prod_datalake_analytics.view_fac_maestro_portafolio_factoring
where codmes = '{mes_incorporar}'
'''
cursor = conn.cursor()
cursor.execute(query)

# Obtener los resultados
resultados = cursor.fetchall()

# Obtener los nombres de las columnas
column_names = [desc[0] for desc in cursor.description]

# Convertir los resultados a un DataFrame de pandas
df_nuevos_datos = pd.DataFrame(resultados, columns=column_names)

################################################################################
from datetime import datetime
from zoneinfo import ZoneInfo
################################################################################
# Hora actual en Perú (UTC-5)
now = datetime.now(ZoneInfo("America/Lima"))

# Guardar directamente el objeto datetime
df_nuevos_datos["_timestamp"] = now
print(now)

# rename de columnas
df_nuevos_datos.columns = columnas_base

# asegurando el tipo de dato
df_nuevos_datos = df_nuevos_datos.astype(dtype_dict)

df_nuevos_datos['cierre_fecha']      = pd.to_datetime(df_nuevos_datos['cierre_fecha']).dt.date
df_nuevos_datos['fecha_desembolso']  = pd.to_datetime(df_nuevos_datos['fecha_desembolso']).dt.date

# Convierte None explícitamente a NaN para que pandas los reconozca
df_nuevos_datos['ultima_fecha_pago'] = df_nuevos_datos['ultima_fecha_pago'].replace({None: pd.NA})
# # Luego convierte a datetime
df_nuevos_datos['ultima_fecha_pago'] = pd.to_datetime(df_nuevos_datos['ultima_fecha_pago'], errors='coerce')
# # Y finalmente deja solo la fecha (sin hora)
df_nuevos_datos['ultima_fecha_pago'] = df_nuevos_datos['ultima_fecha_pago'].dt.date

df_nuevos_datos['fecha_vencimiento'] = pd.to_datetime(df_nuevos_datos['fecha_vencimiento']).dt.date
df_nuevos_datos['fecha']             = pd.to_datetime(df_nuevos_datos['fecha']).dt.date

df_nuevos_datos['_timestamp'] = pd.to_datetime(df_nuevos_datos['_timestamp'])

2026-01-12 16:34:32.230302-05:00


/tmp/ipython-input-476489224.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_nuevos_datos['ultima_fecha_pago'] = pd.to_datetime(df_nuevos_datos['ultima_fecha_pago'], errors='coerce')


In [42]:
df_unido = pd.concat([df_monthly_portfolio,df_nuevos_datos], ignore_index= True)

df_unido = df_unido.sort_values(by = ['codmes'], ascending = [False])

print(df_unido.shape)

(16355, 81)


In [44]:
!pip install pyarrow

#%%
nombre_tabla = 'fac_monthly_portfolio'

# Cliente de S3
s3 = boto3.client(
    "s3",
    aws_access_key_id     = creds["AccessKeyId"],
    aws_secret_access_key = creds["SecretAccessKey"],
    aws_session_token     = creds["SessionToken"],
    region_name           = creds["region_name"]
)

# ==== CONFIGURACIÓN ====
bucket_name = "prod-datalake-sandbox-730335218320"
s3_prefix   = f"{nombre_tabla}/"  # carpeta lógica en el bucket

# ==== EXPORTAR A PARQUET EN MEMORIA ====
parquet_buffer = io.BytesIO()
df_unido.to_parquet(parquet_buffer, index=False, engine="pyarrow")
# también puedes usar engine="fastparquet" si lo prefieres

# Nombre de archivo con timestamp (opcional)
s3_key = f"{s3_prefix}{nombre_tabla}.parquet"

# Subir directamente desde el buffer
s3.put_object(
    Bucket = bucket_name,
    Key    = s3_key,
    Body   = parquet_buffer.getvalue()
)

print(f"✅ Archivo subido a s3://{bucket_name}{s3_key}")

✅ Archivo subido a s3://prod-datalake-sandbox-730335218320fac_monthly_portfolio/fac_monthly_portfolio.parquet
